In [3]:
import networkx as nx
import pandas as pd
import numpy as np

In [11]:
# df = pd.read_csv('../SPREAD_food_networks/sn/results/networks/1.edges', sep=',', usecols = ['source', 'target', 'month'])
df = pd.read_csv('graph.txt', sep = ' ')
def dup_edges(graph_df, delta, period = 12):
    """Duplicate each edge with new timestamps i > period+i""" 
    """CHANGE: now only duplicate edges until period + (delta-1)"""
    # df containing only months to be duplicated (months < delta)
    dup_in_delta = (graph_df.loc[graph_df['month'] < delta])
    
    return pd.concat([graph_df, 
                      pd.concat([dup_in_delta.iloc[:,[0, 1]], 
                                 dup_in_delta.iloc[:,[2]]+period], axis=1)], 
                                 ignore_index=True)
df = dup_edges(df, 3)
print(df)

    source  target  month
0      1.0     4.0      1
1      1.0     4.0      7
2      4.0     2.0      2
3      2.0     4.0      3
4      2.0     4.0      4
5      4.0     2.0      5
6      4.0     2.0      6
7      3.0     5.0      5
8      5.0     3.0      4
9      3.0     6.0      6
10     1.0     4.0     13
11     1.0     4.0     19
12     4.0     2.0     14
13     NaN     NaN     15
14     NaN     NaN     16
15     NaN     NaN     17
16     NaN     NaN     18
17     NaN     NaN     17
18     NaN     NaN     16
19     NaN     NaN     18


In [3]:
G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr='month', create_using=nx.MultiDiGraph)
G = nx.convert_node_labels_to_integers(G, ordering='sorted', label_attribute = 'old_id')
nx.get_node_attributes(G, 'old_id')


{0: 1228208,
 1: 1228209,
 9: 1228217,
 2: 1228210,
 4: 1228212,
 5: 1228213,
 6: 1228214,
 7: 1228215,
 8: 1228216,
 10: 1228218,
 11: 1228219,
 13: 1228221,
 3: 1228211,
 12: 1228220}

In [4]:
list(G.edges(data=True))[0:5]

[(0, 1, {'month': 3}),
 (0, 1, {'month': 4}),
 (0, 1, {'month': 5}),
 (0, 1, {'month': 6}),
 (0, 1, {'month': 15})]

In [5]:
for e in G.edges(data=True):
    e[2]['delta'] = {}

In [6]:
def bidirecCountAndStore(graph, delta):
    """
    graph: a Time-Varying directed network.
    delta: time-interval within which bidirectional edges must be counted.
    for each edge with timestamp t, computes the number of reverse edges within t + delta and stores this in
    its edge attribute.
    Returns the graph with the additional edge attribute
    """
    
    edges = sorted(graph.edges(data=True), key=lambda t: t[2].get('month', 1))
    bidirec_counts = []
    for i in range(len(edges)):
        count = 0
        j = i + 1
        while j < len(edges):
            if edges[j][2]['month'] > edges[i][2]['month'] + delta:
                break
            elif edges[j][0] == edges[i][1] and edges[j][1] == edges[i][0]:
                count += 1
            j += 1
        edges[i][2]['delta'][str(delta)] = count
        bidirec_counts.append(count)
    return edges

In [7]:
G.edges(data=True)
edges = bidirecCountAndStore(G, delta = 4)
# relabel with original node ids
bidirecCountAndStore(G, delta = 2)
G = nx.relabel_nodes(G, nx.get_node_attributes(G, 'old_id'), copy=True)
#list(G.edges(data=True))[0:5]


In [8]:
out_df = nx.to_pandas_edgelist(G, source='source', target='target')
out_df.head()
out_df.to_csv('graph_with_counts.csv')

In [9]:
def get_bidirec_counts(graph, delta, node_pair):
    """
    graph: MultiDiGraph with edge attribute 'delta' and key delta.
    delta: time interval
    node_pair: tuple: check birec edge counts between node pair
    returns total number of bidirectional edges between node pair within given delta.
    """
    count = 0
    for e in graph.edges(data=True):
        if (e[0] == node_pair[0] and e[1] == node_pair[1]) or (e[0] == node_pair[1] and e[1] == node_pair[0]):
            count += e[2]['delta'][str(delta)]
    return count

In [10]:
get_bidirec_counts(G, 4, (1228209,1228219))

196